In [35]:
## start
#print("Hello, World!")

In [1]:
# Import packages
import os
import glob


from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns

import re

#from birdnetlib import Recording
#from birdnetlib.analyzer import Analyzer

#import sqlite3

#import pyarrow.parquet as pq # https://arrow.apache.org/docs/python/parquet.html #error on install

import sys
print(sys.executable)


/opt/anaconda3/envs/nt_bird_webapp_conda_env_20260116/bin/python


In [ ]:
# ==========================================
# 1. DIRECTORY CONFIGURATION
# ==========================================

### Project Root Discovery
SCRIPT_DIR = os.path.dirname(os.path.abspath("__file__")) # dir of this file
PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, "../../")) # root dir of project

### Base Data Directories
RAW_DATA_DIR = os.path.join(PROJECT_ROOT, "nt-bird-detect/data/raw")
PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, "nt-bird-detect/data/processed")
ANALYTICS_DATA_DIR = os.path.join(PROJECT_ROOT, "nt-bird-detect/data/analytics")

# ==========================================
# 2. LOAD RECORDINGS BATCH PARQUET
# ==========================================

monitor_name = "wrangcombe_audio1"

parquet_file_path = os.path.join(PROCESSED_DATA_DIR, monitor_name, "recordings_MASTER.parquet")
df_recordings = pd.read_parquet(parquet_file_path)

# question: For each 24 hour period how many hours does the bird call
df = df_recordings.copy()
df['file_time'] = df['file_time'].astype(str).str.zfill(6)
df['hour'] = df['file_time'].str[:2].astype(int)

df_dates = df.groupby(['file_date']).agg(
        calls=('label', 'count'),              # Total abundance
        hours_active=('hour', 'nunique'),      # Count unique hours
        occupancy_pct=('hour', lambda x: round((x.nunique() / 24) * 100, 1)),  # Occupancy percentage
        confidence=('confidence', 'mean'),      # Mean certainty
        max_confidence=('confidence', 'max')    # Max certainty
    ).reset_index()

df_profiles = df.groupby(['file_date', 'label']).agg(
        calls=('label', 'count'),              # Total abundance
        hours_active=('hour', 'nunique'),      # Count unique hours
        occupancy_pct=('hour', lambda x: round((x.nunique() / 24) * 100, 1)),  # Occupancy percentage
        confidence=('confidence', 'mean'),      # Mean certainty
        max_confidence=('confidence', 'max')    # Max certainty
    ).reset_index()


print(df_dates.head(20))





   file_date  calls  hours_active  occupancy_pct  confidence  max_confidence
0   20250917    268             2            8.3    0.791291        0.997531
1   20250918   3409            13           54.2    0.783028        0.999926
2   20250919   3445            13           54.2    0.772801        0.999887
3   20250920   2207            12           50.0    0.767120        0.999647
4   20250921   3120            13           54.2    0.784609        0.999942
5   20250922   2518            13           54.2    0.773591        0.999967
6   20250923   2820            13           54.2    0.773473        0.999767
7   20250924   3327            13           54.2    0.781361        0.999959
8   20250925   2774            12           50.0    0.748106        0.999962
9   20250926   2902            12           50.0    0.757571        0.999835
10  20250927   2855            12           50.0    0.746889        0.999852
11  20250928   3062            12           50.0    0.751390        0.999595

# Below this cell is old workings

In [ ]:
#### OLD Workings ####
'''
# ==========================================


####
# 20260122
# calculate the occupancy percentage for each species per day

monitor_name = "wrangcombe_audio1"

parquet_file_path = os.path.join(PROCESSED_DATA_DIR, monitor_name, "recordings_MASTER.parquet")
df_recordings = pd.read_parquet(parquet_file_path)

# question: For each 24 hour period how many hours does the bird call
df = df_recordings.copy()
df['file_time'] = df['file_time'].astype(str).str.zfill(6)
df['hour'] = df['file_time'].str[:2].astype(int)

df_profiles = df.groupby(['file_date', 'label']).agg(
        calls=('label', 'count'),              # Total abundance
        hours_active=('hour', 'nunique'),      # Count unique hours
        occupancy_pct=('hour', lambda x: round((x.nunique() / 24) * 100, 1)),  # Occupancy percentage
        confidence=('confidence', 'mean'),      # Mean certainty
        max_confidence=('confidence', 'max')    # Max certainty
    ).reset_index()

print(df_profiles.head())
>
file_date                                       label  calls  hours_active  
0  20250917                  Anas platyrhynchos_Mallard      1             1   
1  20250917     Certhia familiaris_Eurasian Treecreeper      1             1   
2  20250917                  Corvus corone_Carrion Crow      1             1   
3  20250917  Dendrocopos major_Great Spotted Woodpecker      4             2   
4  20250917           Erithacus rubecula_European Robin     93             2   

   occupancy_pct  confidence  max_confidence  
0            4.2    0.695182        0.695182  
1            4.2    0.630476        0.630476  
2            4.2    0.618071        0.618071  
3            8.3    0.899790        0.963596  
4            8.3    0.773933        0.996504




####
# 20260122
# check the daily diversity parquet file
parquet_file_path = os.path.join(ANALYTICS_DATA_DIR, monitor_name, "daily_diversity.parquet")
df_recordings = pd.read_parquet(parquet_file_path)
print(df_recordings.head())
print(df_recordings.info())
>
       date  total_detections  species_richness  shannon_index
0  20250917               268                15           1.90
1  20250918              3409                54           1.53
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              2 non-null      object 
 1   total_detections  2 non-null      int64  
 2   species_richness  2 non-null      int64  
 3   shannon_index     2 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 196.0+ bytes
None


####
# 20260122
# get the total count of bird calls by species, ordered by count

monitor_name = "wrangcombe_audio1"

master_batch_output_path = os.path.join(PROCESSED_DATA_DIR, monitor_name, "recordings_MASTER.parquet")
df_recordings = pd.read_parquet(master_batch_output_path)
df_species_totals = df_recordings.groupby('label').agg(count=('label', 'count')).reset_index()
print(df_species_totals.sort_values(by='count', ascending=False).head())

>
                                       label  count
18         Erithacus rubecula_European Robin   2466
33  Phylloscopus collybita_Common Chiffchaff    325
42                 Regulus regulus_Goldcrest    122
49     Troglodytes troglodytes_Eurasian Wren     89
10       Columba palumbus_Common Wood-Pigeon     72



####
# 20260122
# Find all files to concat for the MASTER parquet file

search_path = os.path.join(PROCESSED_DATA_DIR, monitor_name, "recordings_batch_*.parquet")
daily_files = glob.glob(search_path)

df_list = []
for i in daily_files:
    print(i)
    df_temp = pd.read_parquet(i)
    df_list.append(df_temp)
df_recordings = pd.concat(df_list, ignore_index=True)
print(df_recordings[['file_date']].groupby('file_date').size())
print(df_recordings.tail())
print(df_recordings.info())


# ==========================================
'''